In [18]:
import numpy as np

**Funciones Auxiliares**

In [19]:
def calcularAx(A: np.array, x: np.array):
    res = np.zeros(A.shape[0])  #A.shape[0] devuelve la cantidad de filas de A
    for i, row in enumerate(A):
        for j, value in enumerate(row):
            res[i] += value * x[j][0] 
    return res

In [20]:
def normaInf(A):
    sumatorias = []
    for i, row in enumerate(A):
        sumatorias.append(sum(abs(row)))
    
    return max(sumatorias)

**Modulo ALC**


In [21]:
def matricesIguales(A: np.array, B: np.array, atol = 1e-8):
    if A.size != B.size and A[0].size != B[0].size:
        return False
    for i, fila in enumerate(A):
        for j, valor in enumerate(fila):
           if error(np.float64(valor), np.float64(B[i][j])) > atol:
                return False
    return True

def error(x, y):
    return abs(np.float64(x) - np.float64(y))

def error_relativo(x,y):
    if x == 0:
        return abs(y)
    return error(x,y)/abs(x)


In [22]:
def sonIguales(x,y,atol=1e-08):
    return np.allclose(error(x,y),0,atol=atol)

In [23]:
assert(not sonIguales(1,1.1))
assert(sonIguales(1,1 + np.finfo('float64').eps))
assert(not sonIguales(1,1 + np.finfo('float32').eps))
assert(not sonIguales(np.float16(1),np.float16(1) + np.finfo('float32').eps))
assert(sonIguales(np.float16(1),np.float16(1) + np.finfo('float16').eps,atol=1e-3))

assert(np.allclose(error_relativo(1,1.1),0.1))
assert(np.allclose(error_relativo(2,1),0.5))
assert(np.allclose(error_relativo(-1,-1),0))
assert(np.allclose(error_relativo(1,-1),2))

assert(matricesIguales(np.diag([1,1]),np.eye(2)))
assert(matricesIguales(np.linalg.inv(np.array([[1,2],[3,4]]))@np.array([[1,2],[3,4]]),np.eye(2)))
assert(not matricesIguales(np.array([[1,2],[3,4]]).T,np.array([[1,2],[3,4]])))


In [24]:
def rota(theta: float):
    cos = np.cos(theta)
    sen = np.sin(theta)
    
    return np.array([[cos,-sen],[sen,cos]])

In [25]:
assert np.allclose(rota(0), np.eye(2))

assert np.allclose(
    rota(np.pi / 2),
    np.array([[0, -1],
              [1,  0]])
)

assert np.allclose(
    rota(np.pi),
    np.array([[-1,  0],
              [ 0, -1]])
)


In [26]:
def escala(s):
    matriz = np.eye(len(s))

    for i in range(len(s)):
        matriz[i][i] = s[i]

    return matriz

In [27]:
assert np.allclose(escala([2, 3]), np.array([[2, 0], [0, 3]]))
assert np.allclose(escala([1, 1, 1]), np.eye(3))
assert np.allclose(escala([0.5, 0.25]), np.array([[0.5, 0], [0, 0.25]]))

In [28]:
def rota_y_escala(theta: float, s):
    return escala(s)@rota(theta)


In [29]:
assert np.allclose(
    rota_y_escala(0, [2, 3]),
    np.array([[2, 0],
              [0, 3]])
)

assert np.allclose(
    rota_y_escala(np.pi / 2, [1, 1]),
    np.array([[0, -1],
              [1,  0]])
)

assert np.allclose(
    rota_y_escala(np.pi, [2, 2]),
    np.array([[-2,  0],
              [ 0, -2]])
)


In [30]:
def afin(theta, s, b):
    m1 = rota_y_escala(theta, s)
    return np.array([[m1[0][0],m1[0][1], b[0]],[m1[1][0], m1[1][1], b[1]],[0,0,1]])
    

In [31]:
assert np.allclose(
    afin(0, [1, 1], [1, 2]),
    np.array([
        [1, 0, 1],
        [0, 1, 2],
        [0, 0, 1]
    ])
)

assert np.allclose(
    afin(np.pi / 2, [1, 1], [0, 0]),
    np.array([
        [0, -1, 0],
        [1,  0, 0],
        [0,  0, 1]
    ])
)

assert np.allclose(
    afin(0, [2, 3], [1, 1]),
    np.array([
        [2, 0, 1],
        [0, 3, 1],
        [0, 0, 1]
    ])
)


In [32]:
def trans_afin(v, theta, s, b):
    casi_res = afin(theta, s, b)@np.array([[v[0]],[v[1]],[1]])
    return np.array([casi_res[0][0], casi_res[1][0]])

In [33]:
assert np.allclose(
    trans_afin(np.array([1, 0]), np.pi / 2, [1, 1], [0, 0]),
    np.array([0, 1])
)

assert np.allclose(
    trans_afin(np.array([1, 1]), 0, [2, 3], [0, 0]),
    np.array([2, 3])
)

assert np.allclose(
    trans_afin(np.array([1, 0]), np.pi / 2, [3, 2], [4, 5]),
    np.array([4, 7])  
)


In [34]:
trans_afin(np.array([1, 0]), np.pi / 2, [3, 2], [4, 5])

array([4., 7.])

In [35]:
def norma (x, p):
    if p == 'inf':
        return max(map(abs ,x))
    
    res = 0 
    for xi in x:
        res += xi**p
    return res**(1/p)

In [36]:
assert(np.allclose(norma(np.array([1,1]),2),np.sqrt(2)))
assert(np.allclose(norma(np.array([1]*10),2),np.sqrt(10)))
assert(norma(np.random.rand(10),2)<=np.sqrt(10))
assert(norma(np.random.rand(10),2)>=0)

In [37]:
def normaliza(Xs, p):
    res = []
    for x in Xs:
        res.append(x/norma(x,p))
    return np.array(res, dtype=object)

In [38]:
for x in normaliza([np.array([1]*k) for k in range(1,11)],2):
    assert(np.allclose(norma(x,2),1))
for x in normaliza([np.array([1]*k) for k in range(2,11)],1):
    assert(not np.allclose(norma(x,2),1) )
for x in normaliza([np.random.rand(k) for k in range(1,11)],'inf'):
    assert( np.allclose(norma(x,'inf'),1) )

In [39]:
def normaMatMC(A, q, p, Np):
    m = len(A[0])
    vectors = []
    for i in range(0,Np):
        vectors.append(np.random.rand(m,1))
    
    normalizados = normaliza(vectors, p)

    multiplicados = []
    for vector in normalizados:
        multiplicados.append(calcularAx(A, vector))
    
    
    for i, vector in enumerate(multiplicados):
        multiplicados[i] = (norma(vector, q), normalizados[i])
    
    return max(multiplicados, key=lambda t: t[0])

In [40]:
nMC = normaMatMC(A=np.eye(2),q=2,p=1,Np=100000)
assert(np.allclose(nMC[0],1,atol=1e-3))
assert(np.allclose(np.abs(nMC[1][0]),1,atol=1e-3) or np.allclose(np.abs(nMC[1][1]),1,atol=1e-3))
assert(np.allclose(np.abs(nMC[1][0]),0,atol=1e-3) or np.allclose(np.abs(nMC[1][1]),0,atol=1e-3))

nMC = normaMatMC(A=np.eye(2),q=2,p='inf',Np=100000)
assert(np.allclose(nMC[0],np.sqrt(2),atol=1e-3))
assert(np.allclose(np.abs(nMC[1][0]),1,atol=1e-3) and np.allclose(np.abs(nMC[1][1]),1,atol=1e-3))

A = np.array([[1,2],[3,4]])
nMC = normaMatMC(A=A,q='inf',p='inf',Np=1000000)
assert(np.allclose(nMC[0],normaExacta(A,'inf'),rtol=2e-1)) 

NameError: name 'normaExacta' is not defined

In [41]:
def normaExacta(A, p = [1, 'inf']):
    if p == 1:
        return normaInf(A.T)
    
    elif p == 'inf':
        return normaInf(A)
    
    return None

In [ ]:
assert(np.allclose(normaExacta(np.array([[1,-1],[-1,-1]]),1),2))
assert(np.allclose(normaExacta(np.array([[1,-2],[-3,-4]]),1),6))
assert(np.allclose(normaExacta(np.array([[1,-2],[-3,-4]]),'inf'),7))
assert(normaExacta(np.array([[1,-2],[-3,-4]]),2) is None)
assert(normaExacta(np.random.random((10,10)),1)<=10)
assert(normaExacta(np.random.random((4,4)),'inf')<=4)

In [ ]:
def condMC(A, p, Np=1000000):
    AInv = np.linalg.inv(A)
    normaA = normaMatMC(A, p, p, Np)[0]
    normaAInv = normaMatMC(AInv, p, p, Np)[0]
    

    return normaA * normaAInv

In [ ]:
A = np.array([[1,1],[0,1]])
A_ = np.linalg.solve(A,np.eye(A.shape[0]))
normaA = normaMatMC(A,2,2,10000)
normaA_ = normaMatMC(A_,2,2,10000)
condA = condMC(A,2,10000)
assert(np.allclose(normaA[0]*normaA_[0],condA,atol=1e-3))

A = np.array([[3,2],[4,1]])
A_ = np.linalg.solve(A,np.eye(A.shape[0]))
normaA = normaMatMC(A,2,2,10000)
normaA_ = normaMatMC(A_,2,2,10000)
condA = condMC(A,2,10000)
assert(np.allclose(normaA[0]*normaA_[0],condA,atol=1e-3))


In [ ]:
def condExacta(A, p):
    AInv = np.linalg.inv(A)
    normaA = normaExacta(A, p)
    normaAInv = normaExacta(AInv, p)
    

    return normaA * normaAInv

In [42]:
A = np.random.rand(10,10)
A_ = np.linalg.solve(A,np.eye(A.shape[0]))
normaA = normaExacta(A,1)
normaA_ = normaExacta(A_,1)
condA = condExacta(A,1)
assert(np.allclose(normaA*normaA_,condA))

A = np.random.rand(10,10)
A_ = np.linalg.solve(A,np.eye(A.shape[0]))
normaA = normaExacta(A,'inf')
normaA_ = normaExacta(A_,'inf')
condA = condExacta(A,'inf')
assert(np.allclose(normaA*normaA_,condA))